In [1]:
#!/usr/bin/env python3
import os
import mysql.connector
import pandas as pd

# Update these details with your MySQL configuration
db_config = {
    'user': 'root',            # Your MySQL username
    'password': 'admin',# Your MySQL password
    'host': 'localhost',       # Your MySQL host (usually localhost)
    'database': 'sys',  # The database name you created
    'allow_local_infile': True # Needed for local file import if you use LOAD DATA
}

# Directory where your CSV files are stored
CSV_DIR = r"G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data"

# Connect to the MySQL database
try:
    cnx = mysql.connector.connect(**db_config)
    cursor = cnx.cursor()
    print("Connected to MySQL database.")
except mysql.connector.Error as err:
    print("Error connecting to MySQL:", err)
    exit(1)

# --- Function to load a CSV file and insert the rows into a table ---
def import_csv_to_table(csv_filename, table_name, columns, insert_query):
    file_path = os.path.join(CSV_DIR, csv_filename)
    print(f"Importing {file_path} into {table_name}...")
    df = pd.read_csv(file_path)
    # Ensure the DataFrame has the same columns order as specified
    data = df[columns].values.tolist()
    cursor.executemany(insert_query, data)
    cnx.commit()
    print(f"Imported {len(data)} rows into {table_name}.")

# --- Import data from each CSV file ---
try:
    # 1. Facilities
    fac_columns = ['facility_id', 'facility_name', 'location', 'created_at', 'updated_at']
    fac_insert = "INSERT INTO Facilities (facility_id, facility_name, location, created_at, updated_at) VALUES (%s, %s, %s, %s, %s)"
    import_csv_to_table("Facilities.csv", "Facilities", fac_columns, fac_insert)
    
    # 2. RobotModels
    rm_columns = ['model_id', 'manufacturer', 'model_name', 'model_type', 'payload_capacity', 
                  'reach', 'energy_rating', 'specifications', 'created_at', 'updated_at']
    rm_insert = ("INSERT INTO RobotModels (model_id, manufacturer, model_name, model_type, payload_capacity, "
                 "reach, energy_rating, specifications, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("RobotModels.csv", "RobotModels", rm_columns, rm_insert)
    
    # 3. Robots
    r_columns = ['robot_id', 'facility_id', 'model_id', 'serial_number', 'installation_date', 
                 'last_calibration_date', 'status', 'firmware_version', 'configuration_settings', 'created_at', 'updated_at']
    r_insert = ("INSERT INTO Robots (robot_id, facility_id, model_id, serial_number, installation_date, "
                "last_calibration_date, status, firmware_version, configuration_settings, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Robots.csv", "Robots", r_columns, r_insert)
    
    # 4. Operators
    o_columns = ['operator_id', 'facility_id', 'first_name', 'last_name', 'employee_id', 'role', 
                 'certification_level', 'hire_date', 'contact_info', 'assigned_robot_id', 'created_at', 'updated_at']
    o_insert = ("INSERT INTO Operators (operator_id, facility_id, first_name, last_name, employee_id, role, "
                "certification_level, hire_date, contact_info, assigned_robot_id, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Operators.csv", "Operators", o_columns, o_insert)
    
    # 5. Sensors
    s_columns = ['sensor_id', 'robot_id', 'sensor_name', 'description', 'unit_of_measurement', 
                 'normal_range_min', 'normal_range_max', 'installation_date', 'location_on_robot', 
                 'calibration_date', 'firmware_version', 'status', 'created_at', 'updated_at']
    s_insert = ("INSERT INTO Sensors (sensor_id, robot_id, sensor_name, description, unit_of_measurement, "
                "normal_range_min, normal_range_max, installation_date, location_on_robot, calibration_date, "
                "firmware_version, status, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Sensors.csv", "Sensors", s_columns, s_insert)
    
    # 6. PerformanceMetrics
    pm_columns = ['metric_id', 'robot_id', 'timestamp', 'efficiency', 'energy_consumption', 'cycle_time']
    pm_insert = ("INSERT INTO PerformanceMetrics (metric_id, robot_id, timestamp, efficiency, energy_consumption, cycle_time) "
                 "VALUES (%s, %s, %s, %s, %s, %s)")
    import_csv_to_table("PerformanceMetrics.csv", "PerformanceMetrics", pm_columns, pm_insert)
    
    # 7. EventTypes
    et_columns = ['event_type_id', 'event_name', 'description', 'severity', 'recommended_actions', 
                  'frequency_days', 'estimated_duration', 'created_at', 'updated_at']
    et_insert = ("INSERT INTO EventTypes (event_type_id, event_name, description, severity, recommended_actions, "
                 "frequency_days, estimated_duration, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("EventTypes.csv", "EventTypes", et_columns, et_insert)
    
    # 8. EventLogs
    el_columns = ['event_id', 'robot_id', 'event_type_id', 'event_timestamp', 'details']
    el_insert = ("INSERT INTO EventLogs (event_id, robot_id, event_type_id, event_timestamp, details) "
                 "VALUES (%s, %s, %s, %s, %s)")
    import_csv_to_table("EventLogs.csv", "EventLogs", el_columns, el_insert)
    
    # 9. OperationLogs
    op_columns = ['operation_id', 'robot_id', 'operator_id', 'operation_name', 'start_time', 'end_time', 
                  'cycle_time', 'energy_consumption', 'success', 'output_quantity', 'quality_score', 'notes', 'created_at', 'updated_at']
    op_insert = ("INSERT INTO OperationLogs (operation_id, robot_id, operator_id, operation_name, start_time, end_time, "
                 "cycle_time, energy_consumption, success, output_quantity, quality_score, notes, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("OperationLogs.csv", "OperationLogs", op_columns, op_insert)
    
    # 10. MLModels (if there is data; usually empty)
    ml_columns = ['model_id', 'model_name', 'model_type', 'description', 'version', 'trained_date', 
                  'accuracy_metrics', 'feature_importance', 'hyperparameters', 'created_at', 'updated_at']
    ml_df = pd.read_csv(os.path.join(CSV_DIR, "MLModels.csv"))
    if not ml_df.empty:
        ml_insert = ("INSERT INTO MLModels (model_id, model_name, model_type, description, version, trained_date, accuracy_metrics, "
                     "feature_importance, hyperparameters, created_at, updated_at) "
                     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        ml_data = ml_df[ml_columns].values.tolist()
        cursor.executemany(ml_insert, ml_data)
        cnx.commit()
    
    # 11. AlertConfigurations (if there is data; usually empty)
    ac_columns = ['alert_config_id', 'name', 'description', 'entity_type', 'entity_id', 'condition_type', 
                  'threshold_value', 'comparison_operator', 'severity', 'recipients', 'cooldown_minutes', 
                  'is_active', 'created_at', 'updated_at']
    ac_df = pd.read_csv(os.path.join(CSV_DIR, "AlertConfigurations.csv"))
    if not ac_df.empty:
        ac_insert = ("INSERT INTO AlertConfigurations (alert_config_id, name, description, entity_type, entity_id, condition_type, "
                     "threshold_value, comparison_operator, severity, recipients, cooldown_minutes, is_active, created_at, updated_at) "
                     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        ac_data = ac_df[ac_columns].values.tolist()
        cursor.executemany(ac_insert, ac_data)
        cnx.commit()
    
    print("CSV data imported into MySQL successfully!")
except Exception as e:
    print("Error during import:", e)
finally:
    cursor.close()
    cnx.close()


Connected to MySQL database.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Facilities.csv into Facilities...
Imported 2 rows into Facilities.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\RobotModels.csv into RobotModels...
Imported 6 rows into RobotModels.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Robots.csv into Robots...
Imported 60 rows into Robots.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Operators.csv into Operators...
Error during import: 1054 (42S22): Unknown column 'nan' in 'field list'


In [3]:
#!/usr/bin/env python3
import os
import random
import numpy as np
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Base output directory for CSV files
CSV_DIR = r"G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data"

# MySQL configuration – update with your credentials
db_config = {
    'user': 'root',
    'password': 'admin',  # Update this with your password
    'host': 'localhost',
    'database': 'sys',
    'allow_local_infile': True
}

# Connect to MySQL database
try:
    cnx = mysql.connector.connect(**db_config)
    cursor = cnx.cursor()
    print("Connected to MySQL database.")
except mysql.connector.Error as err:
    print("Error connecting to MySQL:", err)
    exit(1)

# Date range constants
START_DATE = datetime(2021, 1, 1)
END_DATE = datetime(2025, 3, 31)

def random_date(start, end):
    """Return a random datetime between start and end."""
    delta = end - start
    if delta.total_seconds() <= 0:
        return start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

def date_range_years(start_year, end_year, per_year):
    """Generate 'per_year' random dates for each year in the range.
       For the final year, use END_DATE as the upper bound."""
    dates = []
    for year in range(start_year, end_year+1):
        year_start = datetime(year, 1, 1)
        if year == end_year:
            year_end = END_DATE
        else:
            year_end = datetime(year, 12, 31)
        for _ in range(per_year):
            dates.append(random_date(year_start, year_end))
    return dates

def import_csv_to_table(csv_filename, table_name, columns, insert_query):
    """
    Truncate the table first to prevent duplicate inserts.
    Then read the CSV, replace NaN with None, and insert rows.
    """
    file_path = os.path.join(CSV_DIR, csv_filename)
    print(f"Importing {file_path} into {table_name}...")
    
    # Truncate table to remove any existing rows
    cursor.execute(f"TRUNCATE TABLE {table_name}")
    cnx.commit()
    
    df = pd.read_csv(file_path)
    # Replace NaN with None so that MySQL interprets them as NULL
    df = df.where(pd.notnull(df), None)
    # Order the dataframe columns as specified.
    data = df[columns].values.tolist()
    
    cursor.executemany(insert_query, data)
    cnx.commit()
    print(f"Imported {len(data)} rows into {table_name}.")

# --- Import data from each CSV file ---
try:
    # 1. Facilities
    fac_columns = ['facility_id', 'facility_name', 'location', 'created_at', 'updated_at']
    fac_insert = "INSERT INTO Facilities (facility_id, facility_name, location, created_at, updated_at) VALUES (%s, %s, %s, %s, %s)"
    import_csv_to_table("Facilities.csv", "Facilities", fac_columns, fac_insert)
    
    # 2. RobotModels
    rm_columns = ['model_id', 'manufacturer', 'model_name', 'model_type', 'payload_capacity', 
                  'reach', 'energy_rating', 'specifications', 'created_at', 'updated_at']
    rm_insert = ("INSERT INTO RobotModels (model_id, manufacturer, model_name, model_type, payload_capacity, "
                 "reach, energy_rating, specifications, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("RobotModels.csv", "RobotModels", rm_columns, rm_insert)
    
    # 3. Robots
    r_columns = ['robot_id', 'facility_id', 'model_id', 'serial_number', 'installation_date', 
                 'last_calibration_date', 'status', 'firmware_version', 'configuration_settings', 'created_at', 'updated_at']
    r_insert = ("INSERT INTO Robots (robot_id, facility_id, model_id, serial_number, installation_date, "
                "last_calibration_date, status, firmware_version, configuration_settings, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Robots.csv", "Robots", r_columns, r_insert)
    
    # 4. Operators
    o_columns = ['operator_id', 'facility_id', 'first_name', 'last_name', 'employee_id', 'role', 
                 'certification_level', 'hire_date', 'contact_info', 'assigned_robot_id', 'created_at', 'updated_at']
    o_insert = ("INSERT INTO Operators (operator_id, facility_id, first_name, last_name, employee_id, role, "
                "certification_level, hire_date, contact_info, assigned_robot_id, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Operators.csv", "Operators", o_columns, o_insert)
    
    # 5. Sensors
    s_columns = ['sensor_id', 'robot_id', 'sensor_name', 'description', 'unit_of_measurement', 
                 'normal_range_min', 'normal_range_max', 'installation_date', 'location_on_robot', 
                 'calibration_date', 'firmware_version', 'status', 'created_at', 'updated_at']
    s_insert = ("INSERT INTO Sensors (sensor_id, robot_id, sensor_name, description, unit_of_measurement, "
                "normal_range_min, normal_range_max, installation_date, location_on_robot, calibration_date, "
                "firmware_version, status, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Sensors.csv", "Sensors", s_columns, s_insert)
    
    # 6. PerformanceMetrics
    pm_columns = ['metric_id', 'robot_id', 'timestamp', 'efficiency', 'energy_consumption', 'cycle_time']
    pm_insert = ("INSERT INTO PerformanceMetrics (metric_id, robot_id, timestamp, efficiency, energy_consumption, cycle_time) "
                 "VALUES (%s, %s, %s, %s, %s, %s)")
    import_csv_to_table("PerformanceMetrics.csv", "PerformanceMetrics", pm_columns, pm_insert)
    
    # 7. EventTypes
    et_columns = ['event_type_id', 'event_name', 'description', 'severity', 'recommended_actions', 
                  'frequency_days', 'estimated_duration', 'created_at', 'updated_at']
    et_insert = ("INSERT INTO EventTypes (event_type_id, event_name, description, severity, recommended_actions, "
                 "frequency_days, estimated_duration, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("EventTypes.csv", "EventTypes", et_columns, et_insert)
    
    # 8. EventLogs
    el_columns = ['event_id', 'robot_id', 'event_type_id', 'event_timestamp', 'details']
    el_insert = ("INSERT INTO EventLogs (event_id, robot_id, event_type_id, event_timestamp, details) "
                 "VALUES (%s, %s, %s, %s, %s)")
    import_csv_to_table("EventLogs.csv", "EventLogs", el_columns, el_insert)
    
    # 9. OperationLogs
    op_columns = ['operation_id', 'robot_id', 'operator_id', 'operation_name', 'start_time', 'end_time', 
                  'cycle_time', 'energy_consumption', 'success', 'output_quantity', 'quality_score', 'notes', 'created_at', 'updated_at']
    op_insert = ("INSERT INTO OperationLogs (operation_id, robot_id, operator_id, operation_name, start_time, end_time, "
                 "cycle_time, energy_consumption, success, output_quantity, quality_score, notes, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("OperationLogs.csv", "OperationLogs", op_columns, op_insert)
    
    # 10. MLModels (if there is data; typically empty)
    ml_columns = ['model_id', 'model_name', 'model_type', 'description', 'version', 'trained_date', 
                  'accuracy_metrics', 'feature_importance', 'hyperparameters', 'created_at', 'updated_at']
    ml_path = os.path.join(CSV_DIR, "MLModels.csv")
    ml_df = pd.read_csv(ml_path) if os.path.exists(ml_path) else pd.DataFrame()
    ml_df = ml_df.where(pd.notnull(ml_df), None)
    if not ml_df.empty:
        ml_insert = ("INSERT INTO MLModels (model_id, model_name, model_type, description, version, trained_date, accuracy_metrics, "
                     "feature_importance, hyperparameters, created_at, updated_at) "
                     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        ml_data = ml_df[ml_columns].values.tolist()
        cursor.executemany(ml_insert, ml_data)
        cnx.commit()
    
    # 11. AlertConfigurations (if there is data; typically empty)
    ac_columns = ['alert_config_id', 'name', 'description', 'entity_type', 'entity_id', 'condition_type', 
                  'threshold_value', 'comparison_operator', 'severity', 'recipients', 'cooldown_minutes', 
                  'is_active', 'created_at', 'updated_at']
    ac_path = os.path.join(CSV_DIR, "AlertConfigurations.csv")
    ac_df = pd.read_csv(ac_path) if os.path.exists(ac_path) else pd.DataFrame()
    ac_df = ac_df.where(pd.notnull(ac_df), None)
    if not ac_df.empty:
        ac_insert = ("INSERT INTO AlertConfigurations (alert_config_id, name, description, entity_type, entity_id, condition_type, "
                     "threshold_value, comparison_operator, severity, recipients, cooldown_minutes, is_active, created_at, updated_at) "
                     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        ac_data = ac_df[ac_columns].values.tolist()
        cursor.executemany(ac_insert, ac_data)
        cnx.commit()
    
    print("CSV data imported into MySQL successfully!")
except Exception as e:
    print("Error during import:", e)
finally:
    cursor.close()
    cnx.close()


Connected to MySQL database.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Facilities.csv into Facilities...
Error during import: 1701 (42000): Cannot truncate a table referenced in a foreign key constraint (`sys`.`robots`, CONSTRAINT `robots_ibfk_1`)


In [1]:
#!/usr/bin/env python3
import os
import random
import numpy as np
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Base output directory for CSV files
CSV_DIR = r"G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data"

# MySQL configuration
db_config = {
    'user': 'root',
    'password': 'admin',
    'host': 'localhost',
    'database': 'RobotAnalytics',  # Using provided database name
    'allow_local_infile': True
}

# Connect to MySQL database
try:
    cnx = mysql.connector.connect(**db_config)
    cursor = cnx.cursor()
    print("Connected to MySQL database.")
except mysql.connector.Error as err:
    print("Error connecting to MySQL:", err)
    exit(1)

# Date range constants
START_DATE = datetime(2021, 1, 1)
END_DATE = datetime(2025, 3, 31)

def random_date(start, end):
    """Return a random datetime between start and end."""
    delta = end - start
    if delta.total_seconds() <= 0:
        return start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

def date_range_years(start_year, end_year, per_year):
    """Generate 'per_year' random dates for each year in the range.
       For the final year, use END_DATE as the upper bound."""
    dates = []
    for year in range(start_year, end_year+1):
        year_start = datetime(year, 1, 1)
        if year == end_year:
            year_end = END_DATE
        else:
            year_end = datetime(year, 12, 31)
        for _ in range(per_year):
            dates.append(random_date(year_start, year_end))
    return dates

def import_csv_to_table(csv_filename, table_name, columns, insert_query):
    """
    Disable foreign key checks, truncate the table,
    then re-enable foreign key checks, read the CSV,
    replace NaN with None, and insert rows.
    """
    file_path = os.path.join(CSV_DIR, csv_filename)
    print(f"Importing {file_path} into {table_name}...")
    
    # Disable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute(f"TRUNCATE TABLE {table_name}")
    # Re-enable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    cnx.commit()
    
    df = pd.read_csv(file_path)
    # Replace NaN with None so that MySQL interprets them as NULL
    df = df.where(pd.notnull(df), None)
    data = df[columns].values.tolist()
    
    cursor.executemany(insert_query, data)
    cnx.commit()
    print(f"Imported {len(data)} rows into {table_name}.")

# --- Import data from each CSV file ---
try:
    # 1. Facilities
    fac_columns = ['facility_id', 'facility_name', 'location', 'created_at', 'updated_at']
    fac_insert = "INSERT INTO Facilities (facility_id, facility_name, location, created_at, updated_at) VALUES (%s, %s, %s, %s, %s)"
    import_csv_to_table("Facilities.csv", "Facilities", fac_columns, fac_insert)
    
    # 2. RobotModels
    rm_columns = ['model_id', 'manufacturer', 'model_name', 'model_type', 'payload_capacity', 
                  'reach', 'energy_rating', 'specifications', 'created_at', 'updated_at']
    rm_insert = ("INSERT INTO RobotModels (model_id, manufacturer, model_name, model_type, payload_capacity, "
                 "reach, energy_rating, specifications, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("RobotModels.csv", "RobotModels", rm_columns, rm_insert)
    
    # 3. Robots
    r_columns = ['robot_id', 'facility_id', 'model_id', 'serial_number', 'installation_date', 
                 'last_calibration_date', 'status', 'firmware_version', 'configuration_settings', 'created_at', 'updated_at']
    r_insert = ("INSERT INTO Robots (robot_id, facility_id, model_id, serial_number, installation_date, "
                "last_calibration_date, status, firmware_version, configuration_settings, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Robots.csv", "Robots", r_columns, r_insert)
    
    # 4. Operators
    o_columns = ['operator_id', 'facility_id', 'first_name', 'last_name', 'employee_id', 'role', 
                 'certification_level', 'hire_date', 'contact_info', 'assigned_robot_id', 'created_at', 'updated_at']
    o_insert = ("INSERT INTO Operators (operator_id, facility_id, first_name, last_name, employee_id, role, "
                "certification_level, hire_date, contact_info, assigned_robot_id, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Operators.csv", "Operators", o_columns, o_insert)
    
    # 5. Sensors
    s_columns = ['sensor_id', 'robot_id', 'sensor_name', 'description', 'unit_of_measurement', 
                 'normal_range_min', 'normal_range_max', 'installation_date', 'location_on_robot', 
                 'calibration_date', 'firmware_version', 'status', 'created_at', 'updated_at']
    s_insert = ("INSERT INTO Sensors (sensor_id, robot_id, sensor_name, description, unit_of_measurement, "
                "normal_range_min, normal_range_max, installation_date, location_on_robot, calibration_date, "
                "firmware_version, status, created_at, updated_at) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("Sensors.csv", "Sensors", s_columns, s_insert)
    
    # 6. PerformanceMetrics
    pm_columns = ['metric_id', 'robot_id', 'timestamp', 'efficiency', 'energy_consumption', 'cycle_time']
    pm_insert = ("INSERT INTO PerformanceMetrics (metric_id, robot_id, timestamp, efficiency, energy_consumption, cycle_time) "
                 "VALUES (%s, %s, %s, %s, %s, %s)")
    import_csv_to_table("PerformanceMetrics.csv", "PerformanceMetrics", pm_columns, pm_insert)
    
    # 7. EventTypes
    et_columns = ['event_type_id', 'event_name', 'description', 'severity', 'recommended_actions', 
                  'frequency_days', 'estimated_duration', 'created_at', 'updated_at']
    et_insert = ("INSERT INTO EventTypes (event_type_id, event_name, description, severity, recommended_actions, "
                 "frequency_days, estimated_duration, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("EventTypes.csv", "EventTypes", et_columns, et_insert)
    
    # 8. EventLogs
    el_columns = ['event_id', 'robot_id', 'event_type_id', 'event_timestamp', 'details']
    el_insert = ("INSERT INTO EventLogs (event_id, robot_id, event_type_id, event_timestamp, details) "
                 "VALUES (%s, %s, %s, %s, %s)")
    import_csv_to_table("EventLogs.csv", "EventLogs", el_columns, el_insert)
    
    # 9. OperationLogs
    op_columns = ['operation_id', 'robot_id', 'operator_id', 'operation_name', 'start_time', 'end_time', 
                  'cycle_time', 'energy_consumption', 'success', 'output_quantity', 'quality_score', 'notes', 'created_at', 'updated_at']
    op_insert = ("INSERT INTO OperationLogs (operation_id, robot_id, operator_id, operation_name, start_time, end_time, "
                 "cycle_time, energy_consumption, success, output_quantity, quality_score, notes, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    import_csv_to_table("OperationLogs.csv", "OperationLogs", op_columns, op_insert)
    
    # 10. MLModels (if there is data; typically empty)
    ml_columns = ['model_id', 'model_name', 'model_type', 'description', 'version', 'trained_date', 
                  'accuracy_metrics', 'feature_importance', 'hyperparameters', 'created_at', 'updated_at']
    ml_path = os.path.join(CSV_DIR, "MLModels.csv")
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df = ml_df.where(pd.notnull(ml_df), None)
        if not ml_df.empty:
            ml_insert = ("INSERT INTO MLModels (model_id, model_name, model_type, description, version, trained_date, accuracy_metrics, "
                         "feature_importance, hyperparameters, created_at, updated_at) "
                         "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
            ml_data = ml_df[ml_columns].values.tolist()
            cursor.executemany(ml_insert, ml_data)
            cnx.commit()
    
    # 11. AlertConfigurations (if there is data; typically empty)
    ac_columns = ['alert_config_id', 'name', 'description', 'entity_type', 'entity_id', 'condition_type', 
                  'threshold_value', 'comparison_operator', 'severity', 'recipients', 'cooldown_minutes', 
                  'is_active', 'created_at', 'updated_at']
    ac_path = os.path.join(CSV_DIR, "AlertConfigurations.csv")
    if os.path.exists(ac_path):
        ac_df = pd.read_csv(ac_path)
        ac_df = ac_df.where(pd.notnull(ac_df), None)
        if not ac_df.empty:
            ac_insert = ("INSERT INTO AlertConfigurations (alert_config_id, name, description, entity_type, entity_id, condition_type, "
                         "threshold_value, comparison_operator, severity, recipients, cooldown_minutes, is_active, created_at, updated_at) "
                         "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
            ac_data = ac_df[ac_columns].values.tolist()
            cursor.executemany(ac_insert, ac_data)
            cnx.commit()
    
    print("CSV data imported into MySQL successfully!")
except Exception as e:
    print("Error during import:", e)
finally:
    cursor.close()
    cnx.close()


Connected to MySQL database.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Facilities.csv into Facilities...
Imported 2 rows into Facilities.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\RobotModels.csv into RobotModels...
Imported 6 rows into RobotModels.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Robots.csv into Robots...
Imported 60 rows into Robots.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\Project\Data\Operators.csv into Operators...
Imported 120 rows into Operators.
Importing G:\My Drive\Studies\Electrical and Computer Engineering (Robotics and Control)\Winter 2025\Data Management & Applications\